In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
import inspect
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Submit job

In [6]:
from multicamera_airflow_pipeline.tim_240731.interface.o2 import O2Runner
from pathlib import Path
from datetime import datetime

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [7]:
for idx, recording_row in recording_df.iterrows():
    break

In [8]:
recording_row

Subject                                                                  M04002
duration_m                                                                   10
video_recording_id                                     24-05-01-13-26-43-110846
ephys_id                                                    2024-05-01_13-26-37
calibration_id                                         24-05-01-13-45-07-825493
video_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
ephys_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
calibration_location_on_o2    /n/groups/datta/tim_sainburg/datasets/chronic2...
samplerate                                                                  150
username                                                                 tis697
n_ephys_streams                                                               1
max_video_duration_m                                                         10
Name: 0, dtype: object

### Job specific

In [9]:
import yaml
import textwrap
import time

In [10]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline

2D_predictions	    continuous_features   size_normalization  triangulation
arena_alignment     egocentric_alignment  spikesorting	      triangulation2
camera_calibration  gimbal		  tensorrt
camera_sync	    openephys_sync	  tensorrt-2


In [11]:
# these are arguments
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"
job_directory = Path('/n/groups/datta/tim_sainburg/datasets/scratch/jobs')
job_directory.mkdir(exist_ok=True, parents=True)
config_file = Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml")

In [12]:
def convert_minutes_to_hms(minutes_float):
    # Convert minutes to total seconds
    total_seconds = int(minutes_float * 60)
    
    # Extract hours, minutes, and seconds using divmod
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # Format as HH:MM:SS
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [13]:
def check_gimbal_completion(gimbal_output_directory):
    return (gimbal_output_directory / "completed.log").exists()


In [14]:
# load config
config_file = Path(config_file)
config = yaml.safe_load(open(config_file, "r"))

In [15]:
# where the video data is located
recording_directory = Path(recording_row.calibration_location_on_o2) / recording_row.calibration_id

In [16]:
# where to save output
gimbal_output_directory = (
    output_directory / "gimbal" / recording_row.video_recording_id
)
gimbal_output_directory.mkdir(parents=True, exist_ok=True)
current_datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
remote_job_directory = job_directory / current_datetime_str

In [17]:
# check if sync successfully completed
if config["gimbal"]["recompute_completed"] == False:
    if (
        check_gimbal_completion(gimbal_output_directory)
    ):
        print("gimbal completed, quitting")
        # return
    else:
        print("gimbal incomplete, starting")

gimbal completed, quitting


In [18]:
calibration_folder=(
    output_directory
    / "camera_calibration"
    / recording_row.calibration_id
    / "jarvis"
    / "CalibrationParameters"
)

In [19]:
assert calibration_folder.exists()

In [20]:
predictions_3d_directory=(
        output_directory / "triangulation" / recording_row.video_recording_id
    )

In [21]:
assert predictions_3d_directory.exists()

In [22]:
duration_requested = convert_minutes_to_hms(
        recording_row.duration_m * config["o2"]["gimbal"]["o2_runtime_multiplier"]
    )
duration_requested

'01:40:00'

In [23]:
framerate = recording_row.samplerate

In [24]:
params = {
    "gimbal_output_directory": gimbal_output_directory.as_posix(),
    "calibration_folder": calibration_folder.as_posix(),
    "predictions_3d_directory": predictions_3d_directory.as_posix(),
    "samplerate": framerate,
}

In [25]:
# create the job runner
runner = O2Runner(
    job_name_prefix=f"{recording_row.video_recording_id}_gimbal",
    remote_job_directory=remote_job_directory,
    conda_env="/n/groups/datta/tim_sainburg/conda_envs/kpms",
    o2_username=recording_row.username,
    o2_server="login.o2.rc.hms.harvard.edu",
    job_params=params,
    o2_n_cpus=config["o2"]["gimbal"]["o2_n_cpus"],
    o2_memory=config["o2"]["gimbal"]["o2_memory"],
    o2_time_limit=duration_requested,
    o2_queue=config["o2"]["gimbal"]["o2_queue"],
    o2_exclude=config["o2"]["gimbal"]["o2_exclude"],
    o2_qos = config["o2"]["gimbal"]["o2_qos"] ,
    o2_gres = config["o2"]["gimbal"]["o2_gres"] ,
)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Auth banner: b'Problems logging in?\nUse your lower case HMS ID, like abc123, not ABC123.\nIf locked out, see:\nhttps://it.hms.harvard.edu/i-want/reset-password-or-unlock-your-hms-account\n'
INFO:paramiko.transport:Authentication (publickey) successful!


In [26]:
runner.python_script = textwrap.dedent(
f"""
# load params
import yaml
params_file = "{runner.remote_job_directory / f"{runner.job_name}.params.yaml"}"
config_file = "{config_file.as_posix()}"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

from multicamera_airflow_pipeline.tim_240731.keypoints.train_gimbal import GimbalTrainer 
from multicamera_airflow_pipeline.tim_240731.keypoints.inference_gimbal import GimbalInferencer 
# train gimbal
gimbal_trainer = GimbalTrainer(
    gimbal_output_directory=params["gimbal_output_directory"],
    calibration_folder=params["calibration_folder"],
    predictions_3d_directory=params["predictions_3d_directory"],
    samplerate=params["samplerate"],
    **config["gimbal"]["train"]
)
gimbal_trainer.run()

# inference gimbal
gimbal_inferencer = GimbalInferencer(
    gimbal_output_directory=params["gimbal_output_directory"],
    calibration_folder=params["calibration_folder"],
    predictions_3d_directory=params["predictions_3d_directory"],
    **config["gimbal"]["inference"]
)
gimbal_inferencer.run()

"""
)

In [27]:
print(runner.python_script)


# load params
import yaml
params_file = "/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_162449_083799/24-05-01-13-26-43-110846_gimbal_24-08-09-2024-24-49-913403.params.yaml"
config_file = "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

from multicamera_airflow_pipeline.tim_240731.keypoints.train_gimbal import GimbalTrainer 
from multicamera_airflow_pipeline.tim_240731.keypoints.inference_gimbal import GimbalInferencer 
# train gimbal
gimbal_trainer = GimbalTrainer(
    gimbal_output_directory=params["gimbal_output_directory"],
    calibration_folder=params["calibration_folder"],
    predictions_3d_directory=params["predictions_3d_directory"],
    samplerate=params["samplerate"],
    **config["gimbal"]["train"]
)
gimbal_trainer.run()

# inference gimbal
gimbal_inferencer = GimbalInferencer(
    gi

In [28]:
runner.run()

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote job directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_162449_083799
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_162449_083799
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Successfully created remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_162449_083799
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Writing job files to remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_162449_083799
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Transferring /tmp/tmp7mqbxvmz to login.o2.rc.hms.harvard.edu:/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_162449_083799/24-05-01-13-26-43-110846_gimbal_24-08-09-2024-24-49-913403.py
I

In [29]:
# wait until the job is finished
# 10000/60/24 = roughly 1 week
for i in range(10000):
    # check job status every n seconds
    status = runner.check_job_status()
    if status:
        break
    time.sleep(60)

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43759057
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Unknown job state: 
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43759057
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43759057
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43759057
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43759057
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43759057
INFO:multicamera_a

KeyboardInterrupt: 

In [ ]:
# check if sync successfully completed
if check_gimbal_completion(gimbal_output_directory):
    print('gimbal completed successfully')
else:
    raise ValueError("gimbal did not complete successfully.")


In [ ]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_141646_322094/

In [ ]:
!tail /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_141646_322094/24-05-01-13-26-43-110846_gimbal_24-08-09-2024-16-47-240130.log